In [ ]:
!pip install -qU \
    langchain-pinecone==0.1.1 \
    langchain-openai==0.1.8 \
    langchain-text-splitters==0.2.0 \
    langchain==0.2.1 \
    pinecone-notebooks==0.1.1

In [ ]:
import os, getpass
os.environ["PINECONE_API_KEY"] = getpass.getpass("PINECONE_API_KEY:")
api_key = os.environ.get("PINECONE_API_KEY")

In [ ]:
import os, getpass
os.environ["OPENAI_API_KEY"] = getpass.getpass("OPENAI_API_KEY:")

In [ ]:
#7/11
%pip install -qU langchain_community pypdf

In [ ]:
#7/11
from langchain_community.document_loaders import PyPDFLoader

file_path = (
    "/content/resume.pdf"
)

loader = PyPDFLoader(file_path)
pages = loader.load_and_split()

In [ ]:
!pip install --upgrade langchain_openai

In [ ]:
from langchain_openai import OpenAIEmbeddings

model_name = 'text-embedding-3-small'
#model_name = 'text-embedding-3-large'


embeddings = OpenAIEmbeddings(
    model=model_name,
    openai_api_key=os.environ.get('OPENAI_API_KEY')
)

In [ ]:
from pinecone import Pinecone, ServerlessSpec

pc = Pinecone(api_key=os.environ.get("PINECONE_API_KEY"))

In [ ]:

cloud = os.environ.get('PINECONE_CLOUD') or 'aws'
region = os.environ.get('PINECONE_REGION') or 'us-east-1'

spec = ServerlessSpec(cloud=cloud, region=region)

In [ ]:
index_name = "akshu123"

In [ ]:
import time

if index_name not in pc.list_indexes().names():
    pc.create_index(
        name=index_name,
        dimension=1536,
        metric="cosine",
        spec=spec
    )
    # wait for index to be ready
    while not pc.describe_index(index_name).status['ready']:
        time.sleep(1)

In [ ]:
from langchain_pinecone import PineconeVectorStore

namespace = "ns0711"


docsearch = PineconeVectorStore.from_documents(
#    documents=md_header_splits,
    documents=pages, # 07/11
    index_name=index_name,
    embedding=embeddings,
    namespace=namespace
)

In [ ]:
index = pc.Index(index_name)

for ids in index.list(namespace=namespace):
    query = index.query(
        id=ids[0],
        namespace=namespace,
        top_k=1,
        include_values=True,
        include_metadata=True
    )
    print(query)

In [ ]:
index = pc.Index(index_name)
vector_store = PineconeVectorStore(index=index, embedding=embeddings,namespace="ns0711")

vector_store.embeddings

In [ ]:
from langchain_openai import ChatOpenAI
from langchain.chains import RetrievalQA

llm = ChatOpenAI(
    openai_api_key=os.environ.get('OPENAI_API_KEY'),
  #  model_name='gpt-3.5-turbo',
   model_name='gpt-4o-mini',
    temperature=0.0
)

qa = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=vector_store.as_retriever()
)

In [ ]:
query1 = "Where did she do her bachelor's degree?
#multiple queries can be given

In [ ]:
query1_with_knowledge = qa.invoke(query1)
print(query1_with_knowledge["query"],"\n",query1_with_knowledge["result"])
print("\n\n\n")
print('-------2---------')
query1_without_knowledge = llm.invoke(query1)
print(query1_without_knowledge)


In [ ]:
pc.delete_index("akshu123")

In [ ]:
pc.list_indexes()